# SA-OT Model of Jespersen's Cycle Based on Lopopolo & Biro (2011)

## Model Overview
SA-OT (Simulated Annealing for Optimality Theory) extends classical Optimality Theory (OT) -- a model of linguistic competence -- by integrating a stochastic search algorithm that models linguistic performance. 
As such, it relies on the basic components of standard OT:
- a **candidate set** of linguistic forms, and
- a set of **constraints**, ranked into **hierarchies** that represent different grammars.

In addition, it introduces two new elements that allows modeling performance variation:
- a **neighborhood structure (topology)** that organizes the candidate set by structural similarity, and
- a **search algorithm** -- specifically, simulated annealing -- that performs a random walk through this candidate space, guided by a harmony function derived from the constraint hierarchy.

Since the search is stochastic, SA-OT does not always find the globally optimal candidate, i.e., the grammartical form. Instead, it might settle on local optima -- suboptimal outputs, e.g., errors arising from fast speech, slips of the tongue, or tolerable irregularities. This is how SA-OT models linguistic performance, not just idealized grammatical competence thus accounts for linguistic variation and change. 

In [ ]:
from saot import *

# Candidates & Topology

A **candidate** is a pair of underlying form and surface form (uf, sf):
- **uf**: represents the semantics, i.e., the polarity of the utterance -- *positive* or *negative*; 
- **sf**: a *binary syntactic tree*, composed of a main verb (V) and zero or more sentential negation markers (SN). 

Note: We are only interested in negative uf.

In [3]:
for sf in SF_EXAMPLES:    
    print(f"         Original sf: {sf}")
    print(f"Flattened singletons: {flatten_singletons(sf)}")
    print(f"        Flattened sf: {flatten_sf(sf)}")
    print(f"Serialized nested sf: {serialize_sf(sf, flat=False)}")
    print(f"  Serialized flat sf: {serialize_sf(sf, flat=True)}")
    print("-" * 40)

         Original sf: V
Flattened singletons: V
        Flattened sf: ['V']
Serialized nested sf: V
  Serialized flat sf: [V]
----------------------------------------
         Original sf: ['V']
Flattened singletons: V
        Flattened sf: ['V']
Serialized nested sf: V
  Serialized flat sf: [V]
----------------------------------------
         Original sf: ['SN', 'V']
Flattened singletons: ['SN', 'V']
        Flattened sf: ['SN', 'V']
Serialized nested sf: [SN V]
  Serialized flat sf: [SN V]
----------------------------------------
         Original sf: [['V'], 'SN']
Flattened singletons: ['V', 'SN']
        Flattened sf: ['V', 'SN']
Serialized nested sf: [V SN]
  Serialized flat sf: [V SN]
----------------------------------------
         Original sf: [['SN', [['V', 'SN']]]]
Flattened singletons: ['SN', ['V', 'SN']]
        Flattened sf: ['SN', 'V', 'SN']
Serialized nested sf: [SN [V SN]]
  Serialized flat sf: [SN V SN]
----------------------------------------
         Original sf: [

The **neighbors** of a candidate are candidates whose sf's are the results of transforming the original candidate's sf via the following basic steps: 
1. Add an SN at the beginning (left side);
2. Add an SN at the end (right side);
3. Remove the outermost SN;
4. Reverse the order of any daughter nodes (e.g., turn [SN, V] into [V, SN]).

Note: the neighborhood structure (topology) is thus infinite. 

In [ ]:
c = 'V'
c = ['V']
c = ['SN', 'V']
c = [['SN', 'V'], 'SN'] 
c = ['SN', ['V', 'SN']] 

neighbors = generate_neighbors(c)
print("Neighbors of candidate:", serialize_sf(c, flat=False))
for n in neighbors:
    print(" -", serialize_sf(n, flat=False))

Neighbors of candidate: [SN [V SN]]
 - [SN [SN [V SN]]]
 - [[SN [V SN]] SN]
 - [V SN]
 - [SN [SN V]]
 - [[V SN] SN]


Compare with neighborhood space in Figure 3 (p. 30) in Lopolopo and Biro (2011).

# Grammar: Constraints, Hierarchies & K-Values

The grammar is defined based on a set of violable constraints, which express preferences or requirements on surface forms.

There are four constraints, directly based on de Swart (2010):
- **Faith\[Neg]**: The polarity (uf) of the candidate must match the presence or absence of SN in the sf. Assigns 1 violation mark for a mismatch. 
    - Note: Since the input will only be negative polarity, 1 mark is given if there is no SN in the candidate sf. 
- **\*Neg**: Punishes every occurrence of SN in the sf, assigning as many violation marks as number of SN in the sf.
- **NegFirst**: Assigns 1 violation mark if candidate has no preverbal SN.
- **NegLast** (FocusLast in de Swart): Assigns 1 violation mark if candidate has no postverbal SN.

In [ ]:
results = {}

for sf in SF_EXAMPLES:
    violations = eval_constraints(sf)
    results[serialize_sf(sf, flat=True)] = {CONSTRAINTS_LIST[i].__name__: violations[i] for i in range(len(CONSTRAINTS_LIST))}

df = pd.DataFrame.from_dict(results, orient='index')
df = df.fillna(0).astype(int)
print("Constraint violations of candidate sf's (given the input 'uf = negative'):\n (compare with Lopopolo & Biro, 2011, p. 29)")
display(df)

Constraint violations of candidate sf's given the input 'uf = negative':
 (compare with Lopopolo & Biro, 2011, p. 29)


,faith_neg,star_neg,neg_first,neg_last
[V],1,0,1,1
[SN V],0,1,0,1
[V SN],0,1,1,0
[SN V SN],0,2,0,0
[V SN SN],0,2,1,0
[SN SN V],0,2,0,1
[SN V SN SN],0,3,0,0


These constraints are ranked in **hierarchies** and assigned **ranking values (K-values)**, such that the higher the position of a constraint in a hierarchy, the higher its ranking value, and thus the more costly its violation. These constraints rakings represent the possible grammars/language variants.

In [20]:
print("Hierarchies:")
for h_name, h_funcs in HIERARCHIES_DICT.items():
    print(f"\t{h_name}: {' >> '.join([f.__name__ for f in h_funcs])}")

print("-"*40)

print("Constraint k-values:")
print(f"\tDefault k-values: {K_VALUES_DEFAULT}")
print(f"\tRandomly generated k-values between -0.1 and 4.9: {K_VALUES_RANDOM}")

print("-"*40)

print("Example grammar:")
hierarchy_name = 'H1'
k_values = K_VALUES_DEFAULT
grammar = {constraint: k for constraint, k in 
            zip(HIERARCHIES_DICT[hierarchy_name], k_values)}
print(f"\t{grammar_dict_to_readable(grammar)}")

Hierarchies:
	H1: faith_neg >> star_neg >> neg_first >> neg_last
	H2: faith_neg >> neg_first >> star_neg >> neg_last
	H3: faith_neg >> neg_first >> neg_last >> star_neg
	H4: faith_neg >> neg_last >> neg_first >> star_neg
	H5: faith_neg >> neg_last >> star_neg >> neg_first
	H6: faith_neg >> star_neg >> neg_last >> neg_first
----------------------------------------
Constraint k-values:
	Default k-values: [4, 3, 2, 1]
	Randomly generated k-values between -0.1 and 4.9: [4.9, 3.7567185020076264, 3.6292646232092785, 0.13970450993185232]
----------------------------------------
Example grammar:
	{'faith_neg': 4, 'star_neg': 3, 'neg_first': 2, 'neg_last': 1}


# Harmony & SA-OT

In OT, the optimal form given a grammar (i.e., a constraint hierarchy) is determined by the **harmony function**, which compares the constraint violations of the candidates in the candidate set. The candidate which best satisfies the constraints is the one that minimizes the number of violations of higher-ranked constraints. Thus the harmony function in OT always finds the optimal form given the grammar. Note that this requires that the candidate set be finite. 

On the other hand, SA-OT models linguistic performance by searching for the best candidate in the candidate topology given a grammar. Thus, performance emerges from the topology (which may be infinite) and the search algorithm heuristic. The search algorithm used in SA-OT is **simulated annealing (SA)**. It consists in a **random walk** over the topology, starting from one candidate and exploring its neighbors. The search is guided by the harmony function, i.e., how well each form satisfies the grammar (thus the random walk becomes hill climbing where the horizontal component is the search space of candidates defined by the neighborhood function and the vertical component is the harmony function). At each step, the algorithm:
- Picks a random neighbor;
- Compares the harmony of the initial candidate and the chosen neighbor:
	- If the neighbor is equally good or better, move to it;
	- If the neighbor is worse, decide whether to move to it based on a transition probability function, which depends on a temperature parameter.
- Controls the exploration with a temperature parameter:
	- At the start, the temperature is high, so the algorithm explores widely -- even worse options.
	- Over time, the temperature cools, and the algorithm becomes more selective (moves only to better neighbors).
	- Stops when no better neighbors are accepted -> a local optimum has been reached.

In [ ]:
# Example usage of simulated annealing with Optimality Theory

initial_sf = ['V']
hierarchy = HIERARCHIES_DICT['H2']
grammar = {c: K_VALUES_DEFAULT[i] for i, c in enumerate(hierarchy)}
print(f"Grammar: {' >> '.join(c.__name__ + f' (K={K})' for c, K in grammar.items())}")

sa_ot(initial_sf, grammar, K_max=5, K_step=1, t_max=3, t_min=0, t_step=1, max_no_moves=50, seed=42, verbose=True)

Grammar: faith_neg (K=4) >> neg_first (K=3) >> star_neg (K=2) >> neg_last (K=1)

>>> Iteration 1, K=5, t=3, Current: ['V']
Candidate: ['SN', 'V']
Evaluating constraint: faith_neg
    Current: 1, Candidate: 0
Fatal constraint: faith_neg, K-value: 4, Difference: -1
Moved to not-less harmonic neighbor.

>>> Iteration 2, K=5, t=2, Current: ['SN', 'V']
Candidate: ['SN', ['SN', 'V']]
Evaluating constraint: faith_neg
    Current: 0, Candidate: 0
Evaluating constraint: neg_first
    Current: 0, Candidate: 0
Evaluating constraint: star_neg
    Current: 1, Candidate: 2
Fatal constraint: star_neg, K-value: 2, Difference: 1
Moved to less harmonic neighbor because k_C=2 < K=5.

>>> Iteration 3, K=5, t=1, Current: ['SN', ['SN', 'V']]
Candidate: ['SN', 'V']
Evaluating constraint: faith_neg
    Current: 0, Candidate: 0
Evaluating constraint: neg_first
    Current: 0, Candidate: 0
Evaluating constraint: star_neg
    Current: 2, Candidate: 1
Fatal constraint: star_neg, K-value: 2, Difference: -1
Moved t

['SN', 'V']